## 📍 Como usar o mapa acima:

1.  **Navegue:** Use o zoom (scroll do mouse) e arraste o mapa (clicar e segurar) para explorar sua área de interesse.
2.  **Identifique:** Encontre o canal do rio (em azul) onde você deseja definir a saída da bacia (o exutório).
3.  **Posicione o Mouse:** Mova o cursor do mouse exatamente sobre o ponto de saída desejado.
4.  **Copie as Coordenadas:** Olhe no **canto inferior direito do mapa**. Você verá os valores de **(Longitude, Latitude)** mudando em tempo real.
5.  **Cole no Script Principal:** Copie esses dois valores e cole-os na variável `outlet_coords` (na Célula 2) do seu script principal (`1_hand_osmnx.ipynb`).

**Exemplo:**
Se o canto inferior direito mostrar: `(-44.118627, -20.316243)`
No seu outro script, você vai editar a linha para:
`outlet_coords = (-44.118627, -20.316243)`

In [1]:
# # Geração de Insumos Hidrológicos Básicos (Pré-Análise)
#
# Este notebook gera os rasters de **Direção de Fluxo**, **Acumulação de Fluxo** e um vetor da **Rede de Drenagem** para **toda a área do MDE**.
#
# **Objetivo:** Criar os arquivos necessários para carregar em um software SIG (como o QGIS) e permitir a **identificação visual** de um ponto de exutório.
#
# **Arquivos Gerados:**
# 1. `flow_direction.tif`
# 2. `flow_accumulation.tif`
# 3. `rede_drenagem_total.geojson`

# ----------------------------------------------------------------------
# --- CÉLULA 0: Configuração e Bibliotecas Globais ---
# ----------------------------------------------------------------------
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
import warnings
import os

try:
    from pysheds.grid import Grid
except ImportError:
    print("Erro: Biblioteca 'pysheds' não encontrada. Instale com 'pip install pysheds'")
    raise

# Ignorar warnings futuros
warnings.filterwarnings("ignore", category=FutureWarning)
print("Bibliotecas globais importadas com sucesso.")


# ----------------------------------------------------------------------
# --- CÉLULA 1: Parâmetros Globais ---
# ----------------------------------------------------------------------
# Ajuste estes caminhos e valores conforme necessário

# [ENTRADA] Caminho para o seu Modelo Digital de Elevação (MDE/DEM)
INPUT_MDE_PATH = r'srtm_data.tif'

# [SAÍDA] Diretório para salvar os resultados
OUTPUT_DIR = 'resultados_hidrologia_basica'

# [PARÂMETRO] Limiar de acumulação para definir o que é um "rio".
# Ajuste este valor se a rede de drenagem parecer muito densa ou muito esparsa.
# Valores maiores = menos rios (apenas os principais).
# Valores menores = mais rios (incluindo tributários menores).
STREAM_THRESHOLD = 1000

# Cria o diretório de saída se não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Diretório de saída: '{OUTPUT_DIR}'")
print(f"Usando limiar de acumulação: {STREAM_THRESHOLD}")


# ----------------------------------------------------------------------
# --- CÉLULA 2: Cálculo de Direção e Acumulação de Fluxo ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2: Direção e Acumulação de Fluxo ---")

try:
    if not os.path.exists(INPUT_MDE_PATH):
        raise FileNotFoundError(f"Arquivo MDE '{INPUT_MDE_PATH}' não encontrado.")

    print("Carregando MDE e instanciando Grid PySheds...")
    grid = Grid.from_raster(INPUT_MDE_PATH, data_name='dem')
    dem = grid.read_raster(INPUT_MDE_PATH)

    # Condicionamento hidrológico
    print("Condicionando MDE (fill_pits, fill_depressions, resolve_flats)... (Pode demorar)")
    pit_filled_dem = grid.fill_pits(dem)
    flooded_dem = grid.fill_depressions(pit_filled_dem)
    inflated_dem = grid.resolve_flats(flooded_dem) # MDE condicionado

    # Direção e acumulação do fluxo
    print("Calculando direção e acumulação do fluxo... (Pode demorar)")
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
    acc = grid.accumulation(fdir, dirmap=dirmap)

    # Calcular o aspect com NumPy
    print("Calculando o aspect do terreno com NumPy...")
    gy, gx = np.gradient(inflated_dem)
    aspect_rad = np.arctan2(gy, -gx)
    aspect_deg = np.degrees(aspect_rad)
    aspect = (aspect_deg + 360) % 360
    aspect[((gx == 0) & (gy == 0))] = -1 # Marcar áreas planas com -1

    # Salvar rasters intermediários
    fdir_path = os.path.join(OUTPUT_DIR, 'flow_direction.tif')
    acc_path = os.path.join(OUTPUT_DIR, 'flow_accumulation.tif')
    aspect_path = os.path.join(OUTPUT_DIR, 'aspect.tif')

    print(f"Salvando raster de direção de fluxo em '{fdir_path}'...")
    grid.to_raster(fdir, fdir_path, nodata=-9999)

    print(f"Salvando raster de acumulação de fluxo em '{acc_path}'...")
    grid.to_raster(acc, acc_path, nodata=-9999)

    print(f"Salvando raster de aspect em '{aspect_path}'...")
    grid.to_raster(aspect, aspect_path, nodata=-1) # Usar -1 como nodata para o aspect

    print("Célula 2 concluída com sucesso!")
    print("Variáveis `grid`, `fdir`, `acc`, `dirmap`, `inflated_dem` disponíveis para a próxima célula.")

except Exception as e:
    print(f"Erro inesperado na Célula 2: {e}")
    raise

# ----------------------------------------------------------------------
# --- CÉLULA 2.5: Cálculo de Declividade (Slope) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2.5: Cálculo de Declividade (Slope) ---")

# Verificar se a variável da Célula 2 existe
if 'inflated_dem' not in locals():
    print("Erro: Variável 'inflated_dem' da Célula 2 não encontrada. Execute a Célula 2 primeiro.")
    raise NameError("Variável 'inflated_dem' ausente.")

try:
    print("Calculando a declividade (slope) do terreno em graus...")
    # Resolução do raster (tamanho do pixel)
    x_res = grid.viewfinder.affine[0]
    y_res = -grid.viewfinder.affine[4] # É negativo na affine

    # Gradiente
    gy, gx = np.gradient(inflated_dem, y_res, x_res)

    # Magnitude do gradiente
    slope_rad = np.arctan(np.sqrt(gx**2 + gy**2))
    slope_deg = np.degrees(slope_rad)

    # Salvar o raster de declividade
    slope_path = os.path.join(OUTPUT_DIR, 'slope.tif')
    print(f"Salvando raster de declividade em '{slope_path}'...")
    grid.to_raster(slope_deg, slope_path, nodata=-9999)

    print("Célula 2.5 concluída com sucesso!")

except Exception as e:
    print(f"Erro inesperado na Célula 2.5: {e}")
    raise

# ----------------------------------------------------------------------
# --- CÉLULA 2.6: Cálculo do Índice de Umidade Topográfica (TWI) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2.6: Cálculo do Índice de Umidade Topográfica (TWI) ---")

# Verificar se as variáveis necessárias existem
if 'acc' not in locals() or 'inflated_dem' not in locals() or 'grid' not in locals():
    print("Erro: Variáveis da Célula 2 não encontradas. Execute a Célula 2 primeiro.")
    raise NameError("Variáveis da Célula 2 ausentes.")

try:
    print("Calculando o Índice de Umidade Topográfica (TWI)...")
    # Resolução do raster (tamanho do pixel)
    x_res = grid.viewfinder.affine[0]
    y_res = -grid.viewfinder.affine[4]
    cell_area = x_res * y_res

    # Recalcular declividade em radianos
    gy, gx = np.gradient(inflated_dem, y_res, x_res)
    slope_rad = np.arctan(np.sqrt(gx**2 + gy**2))

    # Evitar divisão por zero em áreas planas, substituindo por um valor pequeno
    slope_rad[slope_rad == 0] = 0.001

    # Área de contribuição específica (Specific Catchment Area)
    sca = (acc + 1) * cell_area

    # TWI
    twi = np.log(sca / np.tan(slope_rad))

    # Salvar o raster TWI
    twi_path = os.path.join(OUTPUT_DIR, 'twi.tif')
    print(f"Salvando raster TWI em '{twi_path}'...")
    grid.to_raster(twi, twi_path, nodata=-9999)

    print("Célula 2.6 concluída com sucesso!")

except Exception as e:
    print(f"Erro inesperado na Célula 2.6: {e}")
    raise

# ----------------------------------------------------------------------
# --- CÉLULA 3: Vetorização da Rede de Drenagem (Toda a Área) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 3: Vetorização da Rede de Drenagem ---")

# Verificar se as variáveis da Célula 2 existem
if 'grid' not in locals() or 'fdir' not in locals() or 'acc' not in locals() or 'dirmap' not in locals():
    print("Erro: Variáveis da Célula 2 não encontradas. Execute a Célula 2 primeiro.")
    raise NameError("Variáveis da Célula 2 ausentes.")

try:
    print(f"Criando máscara de rios com limiar > {STREAM_THRESHOLD}...")
    # 1. Máscara de rios (baseada no limiar global)
    streams_mask = acc > STREAM_THRESHOLD

    # 2. EXTRAÇÃO DA REDE VETORIAL
    print("Extraindo a rede de drenagem como feições vetoriais... (Pode demorar)")
    # Nota: Usamos fdir e streams_mask de toda a área, não uma versão recortada
    network = grid.extract_river_network(fdir, streams_mask, distance=1)

    # 3. Conversão para GeoDataFrame
    print("Convertendo feições para GeoDataFrame...")
    features = network.get('features', [])
    geometries = []

    if not features:
        print(f"Aviso: Nenhuma feição de canal encontrada com o limiar de {STREAM_THRESHOLD}.")
        canais_path = None
    else:
        for feature in features:
            coords = feature['geometry']['coordinates']
            # Cria LineString apenas se tiver 2 ou mais pontos
            if len(coords) >= 2:
                line = LineString(coords)
                geometries.append(line)

        if not geometries:
            print("Aviso: Nenhuma geometria de linha válida foi criada.")
            canais_path = None
        else:
            # 5. CRIAR E SALVAR O GEODATAFRAME
            streams_gdf = gpd.GeoDataFrame(geometry=geometries, crs=grid.viewfinder.crs)
            canais_path = os.path.join(OUTPUT_DIR, 'rede_drenagem_total.geojson')
            streams_gdf.to_file(canais_path, driver='GeoJSON')
            print(f"Rede de drenagem (total) salva em: '{canais_path}'")

    print("Célula 3 concluída com sucesso!")
    print("\n--- PROCESSO CONCLUÍDO ---")
    print(f"Arquivos gerados em: '{OUTPUT_DIR}'")

    # --- LISTA DE SAÍDA ATUALIZADA ---
    print(f"1. {os.path.join(OUTPUT_DIR, 'flow_accumulation.tif')}")
    print(f"2. {os.path.join(OUTPUT_DIR, 'flow_direction.tif')}")
    print(f"3. {os.path.join(OUTPUT_DIR, 'aspect.tif')}")
    print(f"4. {os.path.join(OUTPUT_DIR, 'slope.tif')}")
    print(f"5. {os.path.join(OUTPUT_DIR, 'twi.tif')}")
    print(f"6. {canais_path if canais_path else '(Nenhum canal gerado)'}")
    # -----------------------------------

    print("\n>>> Carregue estes arquivos no QGIS para encontrar seu ponto de exutório. <<<")
    print("(Dica: Carregue o 'flow_accumulation.tif' e aplique uma rampa de cores 'Invertida' para ver os rios)")

except Exception as e:
    print(f"Erro inesperado na Célula 3: {e}")
    raise

Bibliotecas globais importadas com sucesso.
Diretório de saída: 'resultados_hidrologia_basica'
Usando limiar de acumulação: 1000

--- Iniciando Célula 2: Direção e Acumulação de Fluxo ---
Carregando MDE e instanciando Grid PySheds...
Condicionando MDE (fill_pits, fill_depressions, resolve_flats)... (Pode demorar)
Calculando direção e acumulação do fluxo... (Pode demorar)
Calculando o aspect do terreno com NumPy...
Salvando raster de direção de fluxo em 'resultados_hidrologia_basica\flow_direction.tif'...
Salvando raster de acumulação de fluxo em 'resultados_hidrologia_basica\flow_accumulation.tif'...
Salvando raster de aspect em 'resultados_hidrologia_basica\aspect.tif'...
Célula 2 concluída com sucesso!
Variáveis `grid`, `fdir`, `acc`, `dirmap`, `inflated_dem` disponíveis para a próxima célula.

--- Iniciando Célula 2.5: Cálculo de Declividade (Slope) ---
Calculando a declividade (slope) do terreno em graus...
Salvando raster de declividade em 'resultados_hidrologia_basica\slope.tif'.